In [1]:
import numpy as np

import sys
sys.path.append("../")
from koopmanlib.dictionary import PsiNN

In [2]:
basis_function = PsiNN(layer_sizes=[100,100,100], n_psi_train=100)

In [3]:
from ou_process import ou_process_1d

theta = 1
mu = 0
sigma = 1
step_num = 10000
h = 1e-5
# X0 = np.random.uniform(-2, 2) # initial point


N = 10000  # Number of the initial points
state = np.random.RandomState()
initial_conditions = state.uniform(-2, 2, size=N)
data_initial = initial_conditions

### Parallel computation ###

# Here we use 8 cores to speed up the computation 
from joblib import Parallel, delayed

def compute_trajectory_1d(initial_conditions):
    X0 = initial_conditions
    X_traj = ou_process_1d(theta, mu, sigma, X0, step_num, h)
    return X_traj[-1]  # Return both initial and final states

# Use 8 cores
n_jobs = 8
results = Parallel(n_jobs=n_jobs)(delayed(compute_trajectory_1d)(ic) for ic in initial_conditions)

# Convert results (which is a list of tuples) into desired format
data_final = np.array(results)  # Shape: (N, 1)

# Pairwise shifted data X and Y
X = data_initial.reshape(N,1) # shape: (10000, 1)
Y = data_final.reshape(N,1)

In [4]:
len_all = X.shape[0]
data_x_train = X[:int(0.7*len_all),:]
data_x_valid = X[int(0.7*len_all)+1:,:]

data_y_train = Y[:int(0.7*len_all),:]
data_y_valid = Y[int(0.7*len_all)+1:,:]

In [5]:
data_train = [data_x_train, data_y_train]
data_valid = [data_x_valid, data_y_valid]

In [6]:
from koopmanlib.solver_0 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-6,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 1s 389ms/step - loss: 0.0097 - val_loss: 0.0097
Epoch 2/2
2/2 [==============================] - 1s 255ms/step - loss: 0.0096 - val_loss: 0.0097
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 1s 272ms/step - loss: 0.0096 - val_loss: 0.0097
Epoch 2/2
2/2 [==============================] - 1s 257ms/step - loss: 0.0096 - val_loss: 0.0097
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 1s 277ms/step - loss: 0.0096 - val_loss: 0.0097
Epoch 2/2
2/2 [==============================] - 1s 255ms/step - loss: 0.0096 - val_loss: 0.0097
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 1s 307ms/step - loss: 0.0096 - val_loss: 0.0097
Epoch 2/2
2/2 [==============================] - 1s 313ms/step - loss: 0.0096 - val_loss: 0.0096
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 1s 296ms/step - loss: 0.0096 - val_loss: 0.0096
Epoch 2/2
2/2 [

In [7]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs0 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs0['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs0['resDMD_DL_outputs']['efuns'].shape)

[ 9.99999321e-01+0.00000000e+00j  9.00485587e-01+0.00000000e+00j
  7.27205971e-01+0.00000000e+00j  5.07805867e-01+0.00000000e+00j
  2.28105497e-01+0.00000000e+00j  8.25559306e-03+0.00000000e+00j
  6.64473361e-04+0.00000000e+00j  9.68626292e-06+0.00000000e+00j
  4.24983137e-07+0.00000000e+00j  1.52912531e-08+0.00000000e+00j
  5.02257243e-10+0.00000000e+00j  2.76211444e-11-3.76670388e-11j
  2.76211444e-11+3.76670388e-11j  1.86042126e-11+5.16182318e-12j
  1.86042126e-11-5.16182318e-12j  1.84243701e-11+1.67833858e-11j
  1.84243701e-11-1.67833858e-11j  1.35579211e-11+1.32976223e-11j
  1.35579211e-11-1.32976223e-11j  1.00090729e-11+2.09156666e-12j
  1.00090729e-11-2.09156666e-12j  8.14544990e-12+2.65476290e-11j
  8.14544990e-12-2.65476290e-11j  6.57150563e-12-8.56517487e-12j
  6.57150563e-12+8.56517487e-12j  6.09139402e-12+3.71189707e-13j
  6.09139402e-12-3.71189707e-13j  4.68257161e-12-1.93451838e-12j
  4.68257161e-12+1.93451838e-12j  3.46862193e-12+5.36258682e-12j
  3.46862193e-12-5.362586

In [8]:
from koopmanlib.solver_1 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-6,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 30s 5s/step - loss: 0.7317 - val_loss: 0.7541
Epoch 2/2
2/2 [==============================] - 1s 335ms/step - loss: 0.7370 - val_loss: 0.7429
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 1s 356ms/step - loss: 0.7312 - val_loss: 0.7371
Epoch 2/2
2/2 [==============================] - 1s 332ms/step - loss: 0.7388 - val_loss: 0.7434
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 1s 348ms/step - loss: 0.7267 - val_loss: 0.7431
Epoch 2/2
2/2 [==============================] - 1s 328ms/step - loss: 0.7323 - val_loss: 0.7359
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 1s 389ms/step - loss: 0.7298 - val_loss: 0.7578
Epoch 2/2
2/2 [==============================] - 1s 347ms/step - loss: 0.7290 - val_loss: 0.7528
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 1s 350ms/step - loss: 0.7325 - val_loss: 0.7437
Epoch 2/2
2/2 [==

In [9]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs1 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs1['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs1['resDMD_DL_outputs']['efuns'].shape)

[ 9.99999349e-01+0.00000000e+00j  9.00483018e-01+0.00000000e+00j
  7.27197476e-01+0.00000000e+00j  5.08421060e-01+0.00000000e+00j
  2.39051253e-01+0.00000000e+00j  8.26741547e-03+0.00000000e+00j
  7.15911707e-04+0.00000000e+00j  9.83000050e-06+0.00000000e+00j
  4.49947536e-07+0.00000000e+00j  1.53760725e-08+0.00000000e+00j
  5.78080130e-10+0.00000000e+00j  3.00746931e-11+0.00000000e+00j
  2.61814562e-11+0.00000000e+00j  1.66812181e-11+9.50508259e-12j
  1.66812181e-11-9.50508259e-12j  1.11459839e-11+0.00000000e+00j
  9.50240337e-12+2.58004438e-11j  9.50240337e-12-2.58004438e-11j
  8.96113321e-12+0.00000000e+00j  8.81387064e-12-5.15703747e-12j
  8.81387064e-12+5.15703747e-12j  8.03498238e-12+1.87225506e-12j
  8.03498238e-12-1.87225506e-12j  5.56294072e-12+2.50147475e-12j
  5.56294072e-12-2.50147475e-12j  5.40280409e-12+1.82040883e-11j
  5.40280409e-12-1.82040883e-11j  3.47368793e-12+4.83964696e-12j
  3.47368793e-12-4.83964696e-12j  2.25585270e-12-1.44869852e-11j
  2.25585270e-12+1.448698

In [10]:
from koopmanlib.solver_2 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-6,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 1s 299ms/step - loss: 0.2868 - val_loss: 0.1569
Epoch 2/2
2/2 [==============================] - 0s 127ms/step - loss: 0.1788 - val_loss: 0.1569
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 139ms/step - loss: 0.3025 - val_loss: 0.1565
Epoch 2/2
2/2 [==============================] - 0s 115ms/step - loss: 0.2794 - val_loss: 0.1563
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 142ms/step - loss: 0.1905 - val_loss: 0.1561
Epoch 2/2
2/2 [==============================] - 0s 118ms/step - loss: 0.2441 - val_loss: 0.1559
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 141ms/step - loss: 0.1708 - val_loss: 0.1557
Epoch 2/2
2/2 [==============================] - 0s 117ms/step - loss: 0.2127 - val_loss: 0.1556
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 0s 138ms/step - loss: 0.3564 - val_loss: 0.1554
Epoch 2/2
2/2 [

In [11]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs2 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs2['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs2['resDMD_DL_outputs']['efuns'].shape)

[ 9.99999140e-01+0.00000000e+00j  9.00441357e-01+0.00000000e+00j
  7.27290985e-01+0.00000000e+00j  5.13372983e-01+0.00000000e+00j
  1.48376406e-01+0.00000000e+00j  7.12756602e-03+0.00000000e+00j
  1.07522945e-03+0.00000000e+00j  8.38403406e-06+0.00000000e+00j
  4.72386997e-07+0.00000000e+00j  1.11932841e-08+0.00000000e+00j
  6.75812452e-10+0.00000000e+00j  4.64168179e-11+0.00000000e+00j
  1.89049921e-11-8.14791154e-12j  1.89049921e-11+8.14791154e-12j
  1.46436282e-11+3.99023267e-11j  1.46436282e-11-3.99023267e-11j
  1.43961912e-11-3.84191681e-12j  1.43961912e-11+3.84191681e-12j
  1.16868703e-11+0.00000000e+00j  1.06813278e-11+1.13453777e-11j
  1.06813278e-11-1.13453777e-11j  9.62128941e-12-4.62676699e-12j
  9.62128941e-12+4.62676699e-12j  8.19883502e-12+1.86021798e-11j
  8.19883502e-12-1.86021798e-11j  6.33642646e-12+7.24797291e-12j
  6.33642646e-12-7.24797291e-12j  5.16011972e-12+0.00000000e+00j
  4.24675463e-12+4.18819369e-12j  4.24675463e-12-4.18819369e-12j
  3.47745016e-12+2.663086

In [12]:
from koopmanlib.solver_3 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-6,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 2s 281ms/step - loss: 3.1771e-07 - val_loss: 1.6062e-07
Epoch 2/2
2/2 [==============================] - 0s 110ms/step - loss: 3.9811e-07 - val_loss: 1.6029e-07
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 138ms/step - loss: 3.0412e-07 - val_loss: 1.6032e-07
Epoch 2/2
2/2 [==============================] - 0s 115ms/step - loss: 4.5298e-07 - val_loss: 1.6050e-07
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 135ms/step - loss: 2.6691e-07 - val_loss: 1.6061e-07
Epoch 2/2
2/2 [==============================] - 0s 113ms/step - loss: 3.6915e-07 - val_loss: 1.6093e-07
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 134ms/step - loss: 2.1197e-07 - val_loss: 1.6076e-07
Epoch 2/2
2/2 [==============================] - 0s 116ms/step - loss: 2.3667e-07 - val_loss: 1.6045e-07
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] -

In [13]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs3 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs3['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs3['resDMD_DL_outputs']['efuns'].shape)

[ 9.99998849e-01+0.00000000e+00j  9.00452138e-01+0.00000000e+00j
  7.27278907e-01+0.00000000e+00j  5.11391500e-01+0.00000000e+00j
  6.15920718e-02+0.00000000e+00j  6.99926576e-03+0.00000000e+00j
  9.82257897e-04+0.00000000e+00j  8.57307310e-06+0.00000000e+00j
  4.95307774e-07+0.00000000e+00j  1.04077887e-08+0.00000000e+00j
  6.67992591e-10+0.00000000e+00j  2.74211322e-11-1.29795269e-11j
  2.74211322e-11+1.29795269e-11j  2.31011092e-11-3.99661728e-11j
  2.31011092e-11+3.99661728e-11j  1.96594667e-11+0.00000000e+00j
  1.54745938e-11-6.74275213e-12j  1.54745938e-11+6.74275213e-12j
  1.18390452e-11-1.49860100e-11j  1.18390452e-11+1.49860100e-11j
  9.87297676e-12-3.36776093e-12j  9.87297676e-12+3.36776093e-12j
  7.89523615e-12+1.56853644e-12j  7.89523615e-12-1.56853644e-12j
  5.23587257e-12+4.56956128e-12j  5.23587257e-12-4.56956128e-12j
  5.17058357e-12-1.53936027e-12j  5.17058357e-12+1.53936027e-12j
  4.13688594e-12-2.34156977e-12j  4.13688594e-12+2.34156977e-12j
  3.36443548e-12-1.491245

In [14]:
from koopmanlib.solver_4 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=1000,
             batch_size=5000,
             lr=1e-6,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 2s 477ms/step - loss: 32317.6551 - val_loss: 799495.0625
Epoch 2/2
2/2 [==============================] - 1s 303ms/step - loss: 28850.3014 - val_loss: 193751.9219
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 1s 344ms/step - loss: 69082.1239 - val_loss: 147756.8750
Epoch 2/2
2/2 [==============================] - 1s 331ms/step - loss: 25509.6808 - val_loss: 199845.9062
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 1s 326ms/step - loss: 49896.7190 - val_loss: 166670.2812
Epoch 2/2
2/2 [==============================] - 1s 306ms/step - loss: 108767.3203 - val_loss: 159043.6562
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 1s 330ms/step - loss: 283121.2433 - val_loss: 195460.0156
Epoch 2/2
2/2 [==============================] - 1s 323ms/step - loss: 78675.5251 - val_loss: 173824.2500
number of the outer loop: 3
Epoch 1/2
2/2 [=======================

In [15]:
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]

resDMD_DL_outputs4 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict
    }
}

print(resDMD_DL_outputs4['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs4['resDMD_DL_outputs']['efuns'].shape)

[ 9.99998845e-01+0.00000000e+00j  9.00458876e-01+0.00000000e+00j
  7.27275844e-01+0.00000000e+00j  5.10402747e-01+0.00000000e+00j
  5.83588676e-02+0.00000000e+00j  6.95497570e-03+0.00000000e+00j
  9.14922412e-04+0.00000000e+00j  8.19476642e-06+0.00000000e+00j
  5.52360953e-07+0.00000000e+00j  9.98728362e-09+0.00000000e+00j
  6.90542024e-10+0.00000000e+00j  8.10727909e-11+0.00000000e+00j
  1.92071884e-11-9.63956061e-12j  1.92071884e-11+9.63956061e-12j
  1.85585562e-11+0.00000000e+00j  1.51339583e-11-1.76007261e-11j
  1.51339583e-11+1.76007261e-11j  1.01804001e-11-5.30110973e-12j
  1.01804001e-11+5.30110973e-12j  9.01694197e-12+3.21427060e-11j
  9.01694197e-12-3.21427060e-11j  8.21904852e-12+1.61896000e-11j
  8.21904852e-12-1.61896000e-11j  7.25474457e-12+8.17676480e-12j
  7.25474457e-12-8.17676480e-12j  6.70716889e-12-3.44719490e-12j
  6.70716889e-12+3.44719490e-12j  5.06470400e-12+0.00000000e+00j
  4.63661619e-12-3.18790437e-12j  4.63661619e-12+3.18790437e-12j
  3.44475758e-12+0.000000